In [137]:

from pathlib import Path
import hashlib
import google.generativeai as genai
import time
import re
import os
import logging

# Configuração da chave da API do Google Generative AI
genai.configure(api_key="sua_chave")

# Configuração de geração de texto
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 18192,
    "candidate_count": 1
}
# Configuração de segurança para evitar conteúdo ofensivo
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
]
# Criação do modelo de geração de texto
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    safety_settings=safety_settings
)
# Lista para armazenar os nomes de animais
nomes_animais=[]

# Função para upload de arquivo
def upload_file(file_path: str) -> str:
    return genai.upload_file(file_path)

# Função para gerar conteúdo de texto
def generate_content(prompt_parts: list) -> str:
    try:
        response = model.generate_content(prompt_parts)
        return re.sub(r'[\'\"\s\n]+', '', response.text)
    except Exception as e:
        logging.error(f"Error generating content: {e}")
        raise

# Função para classificar animais em imagens
def classify_animal(file_path: str) -> str:
    for i in range(0, len(file_path), 5):
        prompt_parts = [
        "Em cada um desses arquivos de imagem tem um animal, escreva para mim qual é a especie desses animais, separando os por '\'':\n",
        "exemplos de saidas:'cachorro','gato','peixe'\n",
        genai.upload_file(file_path[i]),
        genai.upload_file(file_path[i+1]),
        genai.upload_file(file_path[i+2]),
        genai.upload_file(file_path[i+3]),
        genai.upload_file(file_path[i+4]),
        "\n",
    ]
        max_attempts = 7
        for attempt in range(max_attempts):
            try:
                time.sleep(5) 
                return generate_content(prompt_parts)
            except Exception as e:
                logging.error(f"Error classifying animal (attempt {attempt+1}/{max_attempts}): {e}")
                if attempt == max_attempts - 1:
                    raise

# Função para obter lista de arquivos em um diretório
def get_file_paths(directory: str) -> list:
    return [str(file) for file in Path(directory).rglob("*")]

# Diretório de imagens
directory = 'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/bd_imagens'

# Lista de arquivos no diretório
file_list = os.listdir(directory)

# Caminhos de arquivos no diretório
file_paths = get_file_paths(directory)

# Processamento de arquivos em lotes de 5
for i in range(0, len(file_paths), 5):
    file_path=file_paths[i:i+5]
    file_names=file_list[i:i+5]
    try:
        animal_name = classify_animal(file_path)
        nomes_animais.append(animal_name)
        print(animal_name.split(','))
        animais=animal_name.split(',')
        
    except Exception as e:
        logging.error(f"Error processing file {file_path}: {e}")

['raposa', 'caranguejo', 'leão', 'papagaio', 'gato']
['Cachorro', 'Panda', 'Guaxinim', 'Leão', 'Arara']
['lhama', 'gato', 'rolieiro-europeu', 'píton-verde', 'leão']
['Papagaio-do-mar', 'Ovelha', 'Camelo', 'Veado', 'Águia-de-cabeça-branca']


In [139]:
# Criação de lista única de nomes de animais
all_list=[]
for animal in nomes_animais:
    my_list = animal.split(',')
    for lista in my_list:
        all_list.append(lista)

# Diretório de destino para os arquivos classificados
caminho = "C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/"
caminho_arquivo, file_ext = os.path.splitext(file_path[0])

# Processamento de arquivos e renomeação com base nos nomes de animais
for i in range(len(file_paths)):
    file_path, file_ext = os.path.splitext(os.path.join(directory, file_list[i]))

    caminho_pasta=caminho+all_list[i]
    
    if not os.path.exists(caminho_pasta):
        os.makedirs(caminho_pasta)
    os.rename(os.path.join(directory, file_list[i]), os.path.join(caminho_pasta, file_list[i]))



'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/raposa\\alexander-andrews-mEdKuPYJe1I-unsplash.jpg'

'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/caranguejo\\amy-perez-EN24LvCaHw0-unsplash.jpg'

'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/leão\\arleen-wiese-2vbhN2Yjb3A-unsplash.jpg'

'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/papagaio\\brandon-griggs-uhLDvDSNjzs-unsplash.jpg'

'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/gato\\ceyda-ciftci-GGD9KgalPkY-unsplash.jpg'

...

'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/Papagaio-do-mar\\ray-hennessy-mpw37yXc_WQ-unsplash.jpg'

'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/Ovelha\\sam-carter-GHOiyov2TSQ-unsplash.jpg'

'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/Camelo\\sebastian-laube-OkrKzqnNX90-unsplash.jpg'

'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/Veado\\vincent-van-zalinge-WHrwb43vH9E-unsplash.jpg'

'C:/Users/Pablo Vítor/Desktop/Scripts/30daysOfPython/day13/novo_bd/Águia-de-cabeça-branca\\zaur-ibrahimov-vpu5crc2Avc-unsplash.jpg'